In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from repeng.train.train_adapter import evaluate_daily_dilemma, evaluate_model, load_model, load_labels, TrainingConfig, get_choice_ids, select_dilemma_by_values, load_and_process_daily_dilemmas_eval_dataset, process_daily_dilemma_results, proj_root, register_ipissa_peft, setup_adapter, add_adapter_name_to_sd, train_steer_vector
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch
import pandas as pd
import gc

In [3]:

results_dir = sorted(( proj_root / "./outputs/adapters/").glob("*"))
for res_dir in results_dir:
    print(res_dir)

adapter_path = results_dir[-1] / "adapter_model.safetensors"


/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251108_205443
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_062438
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_080729
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_133105


In [4]:
# load
# now evaluate with prompts
config = TrainingConfig()
model_id = config.model_name
base_model, tokenizer = load_model(model_id, quantization_type=config.quantization_type)

# load adapter

# Register InnerPiSSA
register_ipissa_peft()

# Load model and adapter
model = setup_adapter(base_model, config)

# model.model.load_adapter(results_dir)

2025-11-09 15:41:48.178 | INFO     | repeng.train.train_adapter:load_model:220 - Loading model: Qwen/Qwen3-4B-Instruct-2507


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2025-11-09 15:42:04.747 | INFO     | repeng.train.train_adapter:setup_adapter:252 - Adapter configured: rank=64, target_modules=.*\.(5|7|10|13|15|20|25|30|33)\..*(gate_proj|down_proj)


In [5]:
import safetensors.torch
sd = safetensors.torch.load_file(adapter_path)
sd = add_adapter_name_to_sd(sd, adapter_name='honest', prefix="ipissa_")

# now we need to add in .default ?
r = model.load_state_dict(sd, strict=False)
assert not r.unexpected_keys, f"Unexpected keys in state_dict: {r.unexpected_keys[:5]}, missing_keys {r.missing_keys[::5]}"

In [6]:
# now evaluate with prompts
config = TrainingConfig()
model_id = config.model_name
base_model, tokenizer = load_model(model_id, quantization_type=config.quantization_type)


choice_ids = get_choice_ids(tokenizer)
# res, df_res_pv = evaluate_model(model, tokenizer, config, dirs_pca)
generation_config = GenerationConfig(
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    use_cache=True,
    output_logits=True,
    return_dict_in_generate=True,
    do_sample=False,
)


2025-11-09 15:42:04.873 | INFO     | repeng.train.train_adapter:load_model:220 - Loading model: Qwen/Qwen3-4B-Instruct-2507


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2025-11-09 15:42:08.602 | DEBUG    | repeng.train.train_adapter:get_choice_ids:354 - Choice tokens: yes=[',Yes', '.Yes', '_yes', 'eyes', 'Ġyes', 'Yes', 'ĠYES', '.YES', ':YES', 'yes', '_YES', 'ĠYes', '=YES', '=yes', 'YES', '"Yes'], no=['nor', 'INO', 'Ġno', 'ENO', 'Nom', 'ano', '.NO', 'noc', 'ino', 'Nov', '.No', 'ono', 'uno', '_NO', '"No', '/no', 'NOT', '=no', '(no', '.no', 'Uno', 'nov', ',no', 'eno', '-No', 'nop', 'no', 'NON', 'NOW', 'nod', 'nob', '_No', 'ĉno', 'not', 'NOP', 'Nos', 'nom', 'ĠNo', 'NO', '>No', 'ONO', 'Not', 'Non', 'nos', '(NO', '-no', 'No', 'ANO', 'non', ':no', '_no', 'nox', 'Nor', 'now', 'ĠNO', ',No', 'nof', ':NO', 'Now']


In [7]:
df_res_labeled, df_res_pv = evaluate_model(model, tokenizer, config)
df_res_pv.round(3)

2025-11-09 15:42:08.668 | INFO     | repeng.train.train_adapter:evaluate_model:530 - Running evaluation...
2025-11-09 15:42:16.569 | DEBUG    | repeng.train.train_adapter:get_choice_ids:354 - Choice tokens: yes=['yes', 'eyes', '=yes', '_YES', '.Yes', '"Yes', 'Ġyes', '_yes', 'ĠYES', '.YES', 'ĠYes', 'Yes', ':YES', ',Yes', 'YES', '=YES'], no=['INO', 'Nos', '"No', 'nox', ',No', 'No', '=no', 'nod', 'ANO', 'Nov', 'Ġno', ',no', 'ĉno', 'nob', 'nov', 'NOP', 'NON', '/no', 'Non', 'NOW', '_No', 'ONO', ':no', 'no', 'ono', 'non', '>No', '_NO', 'Not', '-no', '.NO', 'eno', '.No', '(NO', 'nos', 'not', 'ino', 'Nom', 'Nor', '(no', 'NO', 'ENO', 'uno', 'Uno', '-No', 'nof', '_no', 'Now', 'noc', 'nom', 'nop', ':NO', 'NOT', 'ĠNo', 'ĠNO', 'now', 'ano', 'nor', '.no']
2025-11-09 15:42:16.570 | INFO     | repeng.train.train_adapter:sweep_coefficients:590 - Evaluating InnerPiSSA (ours) coeff=0 (baseline)
`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True, 'te

eval dd:   0%|          | 0/98 [00:00<?, ?batch/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.01 GiB. GPU 0 has a total capacity of 23.54 GiB of which 1.66 GiB is free. Including non-PyTorch memory, this process has 20.81 GiB memory in use. Of the allocated memory 20.24 GiB is allocated by PyTorch, and 230.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)